## Esercizio 44

Un'industria produce $n$ tipi di elettrodomestici $E=\{1,\dots,n\}$ ed è divisa in $m$ reparti $R=\{1,\dots,m\}$.

Ciascun reparto può fabbricare ciascuno tipo di elettrodomestico. Questa industria dispone di operai ripartiti per reparto. Ciascun operaio ha un monteore prestabilito $M$. Indichiamo con $t_{ji} \ \ i \in E, j\in R$ il tempo di lavorazione (in ore) necessario per ottenere un elettrodomestico pronto per la vendita, per ciascun tipo di elettrodomestico $i$ e per ciascun reparto $j$; mentre indichiamo con $P_i \ \ i\in E$ il prezzo di vendita unitario in euro di ciascun tipo di elettrodomestico.

Questa industria deve pianificare la sua produzione settimanale, deve cioè determinare il numero di ciascuno degli elettrodomestici che deve essere fabbricato da ciascun reparto in modo da soddisfare un ordine di almeno $p_i \ \ i \in E$ elettrodomestici e in modo da massimizzare il profitto complessivo ricavato dalla vendita.


_Attenzione_: ciascun reparto è in grado di fornire prodotti finiti pronti per la vendita, cioè non è necessaria la lavorazione su tutti i reparti per ottenere un prodotto finito!!

I dati dell'istanza sono stati raccolti nel "file ex44.json".

\begin{align}
\max \quad & \sum\limits_{i \in E} P_i \sum\limits_{j \in R} x_{ij} & \\
\quad & \sum\limits_{i \in E} t_{ji} x_{ij} \leq M \cdot o_j & j \in R \\
\quad & \sum\limits_{j \in R} x_{ij} \geq p_i & i \in E\\
\quad & x_{ij} \geq 0 \quad \mbox{integer} & i \in E, j \in R
\end{align}

In [ ]:
# ALERT: execute this cell to prepare input data!
import requests
def download(link, nomeFile=None):
    if nomeFile == None:
        nomeFile = link.split('/')[-1]
    richiesta = requests.get(link)
    if richiesta.status_code == 200:
        with open(nomeFile, 'w') as file:
            file.write(richiesta.text)
            
download('https://tommasoadamo.it/data/ex44.json')

In [ ]:
# ALERT: execute this cell to install DOcplex! 
!pip install docplex cplex

In [ ]:
import docplex.mp.model as cplex
import json

with open('ex44.json', 'r') as f:
    dati = json.load(f)

M = dati['monteore']

n = len(dati['elettrodomestici'])
m = len(dati['reparti'])

P = []
t = dati['tempi']
o = []
p = []

for elettrodomestico in dati['elettrodomestici']:
    P.append(elettrodomestico['prezzo'])
    p.append(elettrodomestico['produzione_minima'])

for reparto in dati['reparti']:
    o.append(reparto['operai'])

E = list(range(n))
R = list(range(m))

with cplex.Model('ex44') as mdl:
    x = mdl.integer_var_matrix(E, R, name='x')
    
    #mdl.maximize(sum(  P[i]*x[i, j]  for i in E for j in R))
    
    mdl.maximize( sum( P[i]*sum(x[i,j] for j in R) for i in E) )
    
    
    for j in R:
        mdl.add_constraint(sum( t[j][i] * x[i,j] for i in E) <= M * o[j], 'cap_'+str(j))
    
    for i in E:
        mdl.add_constraint(sum( x[i,j]  for j in R) >= p[i], 'prod_min_'+str(i) )
    
    mdl.print_information()
    
    sol = mdl.solve()
    
    if sol == None:
        print('No solution')
    else:
        #print(sol)
        for i in E:
            for j in R:
                if x[i,j].solution_value > 0:
                    print('Produci {} di elettrodomestici {} nel reparto {}'.format(x[i,j].solution_value, dati['elettrodomestici'][i]['nome'], dati['reparti'][j]['nome']))